In [1]:
from pairsbacktester import PairsBacktester
from data_processor import DataProcessor

from objects.pairstrategy import PairStrategy
from objects.portfolio import Portfolio

import pandas as pd

from utils import *

In [2]:
# Loading the training data (2017 to 2022)
DATA = pd.read_csv('./data/closing_prices.csv').set_index('Date')

In [3]:
# Get the list of stock tickers
TICKERS = DATA.columns
TICKERS = ['ASIANPAINT', 'BAJFINANCE']

# Split training data (2017 to 2022) from the corpus

TRAINING_DATA = DATA[:'2023-01-01']
TESTING_DATA = DATA['2023-01-01':]

# Store all the cointegrated pairs in this list
COINT_PAIRS = []

for first_ticker in range(len(TICKERS) - 1):
    for second_ticker in range(first_ticker + 1, len(TICKERS)):

        ticker1 = TICKERS[first_ticker]
        ticker2 = TICKERS[second_ticker]

        data1 = TRAINING_DATA[ticker1]
        data2 = TRAINING_DATA[ticker2]

        # Retrieve model 
        model = get_regression_model(data1, data2)

        # Get residuals to test for stationarity
        residuals = model.resid

        # Gauge cointegration by performing the AdFuller test with a confidence level of 99%
        if passes_adfuller_test(residuals, alpha=0.01):
            # If test passes then add the pair to the list
            print (f'Pair ({ticker1}, {ticker2}) is cointegrated (99% confidence)')
            COINT_PAIRS.append({
                'pair': str((ticker1, ticker2)),
                'ticker1': ticker1,
                'ticker2': ticker2,
                'model': model
            })

model = COINT_PAIRS[0]['model']

closing_prices = TESTING_DATA[['ASIANPAINT', 'BAJFINANCE']]
ticker1 = closing_prices['ASIANPAINT']
ticker2 = closing_prices['BAJFINANCE']

data = DataProcessor.get_processed_data(
    pair_tickers=['ASIANPAINT', 'BAJFINANCE'],
    pair_closing_prices=closing_prices,
    pair_regression_model=model
)

Pair (ASIANPAINT, BAJFINANCE) is cointegrated (99% confidence)
Date
2023-01-02   -0.059694
2023-01-03    0.141535
2023-01-04    0.127458
2023-01-05   -0.761089
2023-01-06   -0.872727
                ...   
2023-12-22   -0.035266
2023-12-26   -0.514567
2023-12-27   -0.477895
2023-12-28   -0.389612
2023-12-29   -0.275854
Name: ASIANPAINTBAJFINANCE_residual, Length: 245, dtype: float64


In [4]:
data

,ASIANPAINT,BAJFINANCE,ASIANPAINTBAJFINANCE_residual,date,ASIANPAINTBAJFINANCE_prev_residual
Date,,,,,
2023-01-02,3047.250000,6553.750000,-0.059694,2023-01-02,NaN
2023-01-03,3028.250000,6606.250000,0.141535,2023-01-03,-0.059694
2023-01-04,3016.850098,6571.000000,0.127458,2023-01-04,0.141535
2023-01-05,3004.350098,6100.049805,-0.761089,2023-01-05,0.127458
2023-01-06,2978.399902,5980.399902,-0.872727,2023-01-06,-0.761089
...,...,...,...,...,...
2023-12-22,3341.300049,7295.299805,-0.035266,2023-12-22,0.301994
2023-12-26,3383.350098,7162.299805,-0.514567,2023-12-26,-0.035266
2023-12-27,3404.449951,7232.799805,-0.477895,2023-12-27,-0.514567


In [ ]:
strategy = PairStrategy(
    id='1'
    ticker1='ASIANPAINT',
    ticker2='BAJFINANCE',
    entry_condition_z_value=2,
    target_condition_z_value=0,
    stoploss_condition_z_value=3,
    target_perc=0.1,
    stoploss_perc=-0.02
)